In [17]:
!pip install ISLP

1. Standard Formula with sm.GLM()

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import ModelSpec, summarize

# Set random seed

In [ ]:

np.random.seed(42)

# Load Default data

In [ ]:

Default = load_data('Default')

# Prepare X and y

In [27]:

y = Default['default'] == 'Yes'
X = ModelSpec(['income', 'balance']).fit_transform(Default)

# Fit logistic regression

In [28]:

glm = sm.GLM(y, X, family=sm.families.Binomial())
result = glm.fit()

# Display summary and standard errors

In [29]:
print(summarize(result))
print("\nStandard errors using standard formula:")
print(result.bse)

                coef   std err       z  P>|z|
intercept -11.540500  0.435000 -26.544    0.0
income      0.000021  0.000005   4.174    0.0
balance     0.005600  0.000000  24.835    0.0

Standard errors using standard formula:
intercept    0.434772
income       0.000005
balance      0.000227
dtype: float64


2. Bootstrap Estimates for Standard Error

In [ ]:
def bootfn(data, idx):
    D = data.iloc[idx]
    y_boot = D['default'] == 'Yes'
    X_boot = ModelSpec(['income', 'balance']).fit_transform(D)
    glm_boot = sm.GLM(y_boot, X_boot, family=sm.families.Binomial())
    res_boot = glm_boot.fit()
    return res_boot.params[1:]  # [income, balance]

In [31]:
B = 1000
n = len(Default)
boot_coefs = np.zeros((B, 2))

In [ ]:
np.random.seed(42)
for i in range(B):
    idx = np.random.choice(n, n, replace=True)
    boot_coefs[i, :] = bootfn(Default, idx)

In [32]:
bootstrap_se = boot_coefs.std(axis=0)
print("\nBootstrap standard errors (income, balance):", bootstrap_se)


Bootstrap standard errors (income, balance): [0. 0.]


The first method directly retrieves standard errors calculated analytically by statsmodels.

The second method resamples the dataset 1000 times (with replacement), fits the model each time, and calculates the empirical (sample) standard deviation of those estimates as the bootstrap standard error.


*************************************************************************************************************

(a) Using the summarize() and sm.GLM() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors. 


In [33]:
glm = sm.GLM(y, X, family=sm.families.Binomial())
result = glm.fit()


In [ ]:
# Display regression summary including standard errors
print(summarize(result))
print("\nStandard errors for income and balance:")
print(result.bse[1:])  

                coef   std err       z  P>|z|
intercept -11.540500  0.435000 -26.544    0.0
income      0.000021  0.000005   4.174    0.0
balance     0.005600  0.000000  24.835    0.0

Standard errors for income and balance:
income     0.000005
balance    0.000227
dtype: float64


The summarize() function provides a neat summary of regression output, including the estimated standard errors.

result.bse gives the standard errors for all coefficients; use result.bse[1:] for 'income' and 'balance'.


****

(b) Write a function, boot_fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefcient estimates for income and balance in the multiple logistic regression model. 

In [37]:
import statsmodels.api as sm
from ISLP.models import ModelSpec

In [48]:
def boot_fn(data, idx):
    # Subset the data based on given indices
    sample = data.iloc[idx]
    y = sample['default'] == 'Yes'
    X = ModelSpec(['income', 'balance']).fit_transform(sample)
    model = sm.GLM(y, X, family=sm.families.Binomial())
    result = model.fit()
    return result.params[['income', 'balance']]

This function subsets the Default data using the given index, fits a logistic regression model, and returns the coefficients for income and balance predictors.


***

(c) Following the bootstrap example in the lab, use your boot_fn() function to estimate the standard errors of the logistic regression coefcients for income and balance. 

In [ ]:
import numpy as np
from ISLP import load_data

In [50]:
# Load Default data
Default = load_data('Default')

In [51]:
# Set seed for reproducibility
np.random.seed(42)


In [52]:
# Number of bootstrap samples
B = 1000

# Create an array to store the bootstrap estimates (for income and balance)
boot_coefs = np.zeros((B, 2))


In [ ]:
# Bootstrap process
for i in range(B):
    # Generate bootstrap sample indices
    indices = np.random.choice(len(Default), len(Default), replace=True)
    # Estimate coefficients with bootstrap sample
    coefs = boot_fn(Default, indices)
    boot_coefs[i, :] = coefs.values

In [53]:
# Calculate bootstrap standard errors as the standard deviation across bootstrap samples
bootstrap_se = boot_coefs.std(axis=0)
print("Bootstrap standard errors for income and balance:", bootstrap_se)

Bootstrap standard errors for income and balance: [0. 0.]


(d) Comment on the estimated standard errors obtained using the sm.GLM() function and using the bootstrap.

The standard errors from sm.GLM() and the bootstrap both estimate how much the coefficient values might change if we took new samples.

sm.GLM() standard errors use math formulas based on the model assumptions. They are quick to calculate.

Bootstrap standard errors come from actually resampling the data many times and refitting the model. They don’t assume the model is perfect and can be more reliable if the data is unusual.

If both are close, it means the model assumptions are probably fine. If they differ a lot, it might mean the model needs a closer look. The bootstrap is more flexible but takes more computing power.

***

2.	We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis

(a) Fit a logistic regression model that uses income and balance to predict default.

In [54]:
import numpy as np
from ISLP import load_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [55]:
# Load Default dataset
Default = load_data('Default')

In [56]:
# Prepare response variable and predictors
y = Default['default'] == 'Yes'
X = ModelSpec(['income', 'balance']).fit_transform(Default)

In [62]:
# Fit logistic regression model
glm = sm.GLM(y, X, family=sm.families.Binomial())
result = glm.fit()

In [63]:
# Display model summary
print(summarize(result))

                coef   std err       z  P>|z|
intercept -11.540500  0.435000 -26.544    0.0
income      0.000021  0.000005   4.174    0.0
balance     0.005600  0.000000  24.835    0.0


(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps: 


(a) Fit a logistic regression model that uses income and balance to predict default. 
(b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps: 
i. Split the sample set into a training set and a validation set. 
ii. Fit a multiple logistic regression model using only the training observations. 
iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5. 

iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified. 
(c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Comment on the results obtained.


In [71]:
import numpy as np
from ISLP import load_data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [72]:

# Load Default data
Default = load_data('Default')

In [73]:
# Prepare predictors and response
X = Default[['income', 'balance']]
y = (Default['default'] == 'Yes').astype(int)

validation_errors = []

In [74]:
# Repeat splitting, training, predicting, and evaluating 3 times
for seed in [1, 2, 3]:
    np.random.seed(seed)  # Set random seed

In [ ]:
  
    # i. Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=seed)

In [79]:
 
    # ii. Fit logistic regression on training set
model = LogisticRegression()
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [78]:

    # iii. Predict posterior probabilities on validation set
prob_pred = model.predict_proba(X_val)[:, 1]

In [77]:
# Classify default if probability > 0.5
y_pred = (prob_pred > 0.5).astype(int)
    
    # iv. Compute validation set error (fraction misclassified)
validation_error = np.mean(y_pred != y_val)
validation_errors.append(validation_error)
print(f"Validation error for split {seed}: {validation_error:.4f}")



Validation error for split 3: 0.0482


In [80]:
# Comment on results
print("\nComment: The validation errors vary a little between trials because of random differences in training and validation sets. Repeating with different samples ensures a more reliable estimate of test error by accounting for this variability.")



Comment: The validation errors vary a little between trials because of random differences in training and validation sets. Repeating with different samples ensures a more reliable estimate of test error by accounting for this variability.


The logistic regression model uses income and balance to predict default.

The sample is split into training and validation sets for each trial.

The model is trained on training data only, then predictions are made on validation data.

Validation error is computed as the fraction of misclassified validation samples.

The process is repeated 3 times with different splits.

The variability in validation errors across repeats reflects sampling variability and the importance of multiple validation splits.